## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-03-15-50-15 +0000,bbc,"Rayner admits underpaying tax on £800,000 Hove...",https://www.bbc.com/news/articles/cy50446rq73o...
1,2025-09-03-15-49-00 +0000,wsj,"Opinion | Trump, Lisa Cook and the Federal Res...",https://www.wsj.com/opinion/trump-lisa-cook-an...
2,2025-09-03-15-47-00 +0000,wsj,Opinion | Newsom Escapes His Own Censorship Law,https://www.wsj.com/opinion/newsom-escapes-his...
3,2025-09-03-15-46-56 +0000,nypost,"Horrified kids shout ‘Chuck E, No!’ as cops cu...",https://nypost.com/2025/09/03/us-news/horrifie...
4,2025-09-03-15-44-36 +0000,nyt,"After Graham Linehan’s Arrest, Police Chief Sa...",https://www.nytimes.com/2025/09/03/world/europ...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2311/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
6,trump,51
43,china,25
253,kim,18
465,military,15
86,putin,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
148,2025-09-03-05-07-00 +0000,nypost,"Trump accuses Putin, Kim Jong Un and Xi Jinpin...",https://nypost.com/2025/09/03/us-news/trump-ac...,180
164,2025-09-03-02-57-51 +0000,nypost,China’s Xi hosts largest-ever military parade ...,https://nypost.com/2025/09/02/world-news/china...,121
140,2025-09-03-06-21-08 +0000,bbc,'A clear message to Trump': BBC correspondents...,https://www.bbc.com/news/articles/c7545p2px5no...,110
155,2025-09-03-03-45-58 +0000,bbc,Trump accuses Xi of conspiring against US with...,https://www.bbc.com/news/articles/c1dxd9eyy71o...,107
168,2025-09-03-02-20-57 +0000,bbc,"Moment Xi, Putin and Kim pose with world leade...",https://www.bbc.com/news/videos/cly6ylk070zo?a...,105


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
148,180,2025-09-03-05-07-00 +0000,nypost,"Trump accuses Putin, Kim Jong Un and Xi Jinpin...",https://nypost.com/2025/09/03/us-news/trump-ac...
145,56,2025-09-03-05-23-07 +0000,nypost,Vice President JD Vance and second lady Usha w...,https://nypost.com/2025/09/03/us-news/vice-pre...
204,49,2025-09-02-23-16-19 +0000,nypost,Dramatic footage shows US strike that oblitera...,https://nypost.com/2025/09/02/world-news/foota...
193,41,2025-09-03-00-03-52 +0000,latimes,Cardi B wins civil assault trial brought by se...,https://www.latimes.com/entertainment-arts/sto...
4,41,2025-09-03-15-44-36 +0000,nyt,"After Graham Linehan’s Arrest, Police Chief Sa...",https://www.nytimes.com/2025/09/03/world/europ...
12,40,2025-09-03-15-22-00 +0000,wsj,Several of Epstein’s victims stepped into the ...,https://www.wsj.com/politics/policy/epstein-vi...
308,34,2025-09-02-17-48-32 +0000,bbc,Trump's use of National Guard in Los Angeles w...,https://www.bbc.com/news/articles/c5ylyd9lkkqo...
217,34,2025-09-02-22-18-35 +0000,nypost,Six candidates from Germany’s right-wing AfD p...,https://nypost.com/2025/09/02/world-news/six-c...
35,32,2025-09-03-14-20-52 +0000,latimes,Wildfire burns into historic California Gold R...,https://www.latimes.com/california/story/2025-...
225,31,2025-09-02-21-50-13 +0000,nypost,Google dodges forced selloff of Chrome browser...,https://nypost.com/2025/09/02/business/google-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
